<a href="https://colab.research.google.com/github/digitalblue/aaie-model-lab/blob/document%2Fprompting-experiment-plan/AAIE_Prompting_Design_Qwen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AAIE - Prompting Design - Qwen
---
This notebook:
* Uses langchain for inference on open model (Qwen)
* Loads and parses Json rubric into string format for prompting
* Time duration of LLM response

In [1]:
!pip install langchain-huggingface transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface.llms import HuggingFacePipeline
from google.colab import userdata
import json
import os
import time

### Load the LLM model from Huggingface
Update the model id value to load other open models from huggingface model hub.

In [3]:
MODEL_ID = "Qwen/Qwen3-0.6B"

In [15]:
hf = HuggingFacePipeline.from_model_id(
    model_id=MODEL_ID,
    task="text-generation",
    pipeline_kwargs={"temperature": 0.7},
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


### System Prompt

In [5]:
system_prompt = """
You are a helpful and respectful educational assessment assistant that provides feedback on submitted work.
Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Provide assessment feedback and a rating for the assessment based on performce descriptors.
Ensure your assessment is accurate, do not guess or make assumptions.
If you something is unclear seek clarification from the student.
"""

### Load and formatting functions

In [4]:
def load_rubric_from_json(json_path):
    with open(json_path, "r") as f:
        rubric = json.load(f)
    return rubric

In [9]:
def get_prompt_template(user_prompt):
    prompt_template = ChatPromptTemplate([
        ("system", system_prompt),
        ("user", user_prompt)
    ])
    return prompt_template

In [10]:
def construct_user_prompt(domain, rubric, submission):
  user_prompt = f"""
  Given a submission in domain of {domain}, provide an assessment using this rubric:
  {rubric}
  This is the submission:
  {submission}
   """
  return user_prompt

In [11]:
def format_rubric(rubric):
  formatted_rubric = f"""
  Rubric ID: {rubric['rubric_id']}

  Criteria:
  """

  for rubric_item in rubric['criteria']:
    formatted_rubric += f"""
    Criterion: {rubric_item['criterion_id']}
    Name: {rubric_item['name']}
    Description: {rubric_item['description']}
    Performance Descriptors:
    """
    for key, val in rubric_item['performance_descriptors'].items():
      formatted_rubric += f"""
      - {key}: {val}
      """
  return formatted_rubric

### Load rubric and construct prompt
Change file name and path to test other rubrics from data team

In [7]:
rubric = load_rubric_from_json("rub_psychology_0011.json")

In [8]:
rubric.keys()

dict_keys(['domain', 'prompt', 'rubric', 'submissions'])

In [13]:
user_prompt = construct_user_prompt("psychology", format_rubric(rubric['rubric']), rubric['submissions'][0]['final_submission'])

In [14]:
prompt = get_prompt_template(user_prompt)

### Get model response

In [16]:
t1 = time.time()
response = hf.invoke(prompt.format())
t2 = time.time()
print(f"Time taken: {t2-t1} seconds")

Time taken: 12.473321914672852 seconds


In [17]:
print(response)

System: 
You are a helpful and respectful educational assessment assistant that provides feedback on submitted work.
Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Provide assessment feedback and a rating for the assessment based on performce descriptors.
Ensure your assessment is accurate, do not guess or make assumptions.
If you something is unclear seek clarification from the student.

Human: 
  Given a submission in domain of psychology, provide an assessment using this rubric:
  
  Rubric ID: rub_psychology_0011

  Criteria:
  
    Criterion: c1
    Name: Theoretical Understanding
    Description: Demonstrates comprehension of relevant psychological theories and their application.
    Performance Descriptors:
    
      - excellent: Demonstrates comprehensive and nuanced understanding of complex psychological theories
      
      - good: Shows solid understandi